In [1]:
## GeNNus Simone & Paolo

In [2]:
import torch
import tqdm
import os
import numpy as np
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import random 
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

import pandas as pd

In [3]:
DATASET_SIZE = "xxxxs"
DATASET_TYPE = "waveform"

DATASET_FOLDER = f"./data/{DATASET_TYPE}"

DATASET_NUM_SAMPLES_PER_SECOND = 8000
DATASET_NUM_CHANNELS = 1

DATASET_NAME = f"fma_{DATASET_SIZE}_resampled_{DATASET_NUM_SAMPLES_PER_SECOND}_rechanneled_{DATASET_NUM_CHANNELS}"

dataset_path = f"{DATASET_FOLDER}/{DATASET_NAME}"

In [4]:
audio_path_list = []

for path, subdirs, files in os.walk(dataset_path):
    for name in files:
        file_audio_path = os.path.join(path, name)
        print(file_audio_path)

        if name != '.DS_Store':
            audio_path_list.append(file_audio_path)

audio_path_list = sorted(audio_path_list , reverse= True)
# lista di tutti i path     

./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070321.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070314.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070319.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070317.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070324.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070322.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070325.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070320.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070315.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070316.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070328.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070327.waveform
./data/waveform/fma_xxxxs_resampled_8000_rechanneled_1/Pop/070326.waveform
./data/waveform/fma_xxxxs

In [5]:
single_tensors = []
labels = []

for p in audio_path_list:
    single_tensors.append(torch.load(p))
    #print(p)
    labels.append(p.split("/")[-2])

single_tensors # --> waveforms
labels # --> etichette

['Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Rock',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Jazz',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Hip-Hop',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic']

In [6]:
stacked_single_tensors = torch.cat(single_tensors).numpy()

In [7]:
data = pd.DataFrame(stacked_single_tensors)
data.insert(0, "Label", labels, True)

In [8]:

def train_test_split( dataset , perc_train , set_seed = 69):

    train = dataset.sample(frac= perc_train,random_state = set_seed )
    test  = dataset.drop(train.index)

    return (train , test )


In [9]:
train_dataset , test_dataset = train_test_split( data , perc_train = .9)

In [10]:

X = train_dataset.iloc[: , 1:]
y = train_dataset.iloc[: , :1]

std_slc = StandardScaler()

pca = decomposition.PCA()

logistic_Reg = linear_model.LogisticRegression()


pipe = Pipeline(steps=[('std_slc', std_slc),
                ('pca', pca),
                ('logistic_Reg', logistic_Reg)])


n_components = list(range( int(.65 * X.shape[0]) ,  int(.75 * X.shape[0])    , 1))


C = [2., 20.]

penalty = ['l1', 'l2']


parameters = dict(pca__n_components=n_components,
                    logistic_Reg__C=C,
                    logistic_Reg__penalty=penalty)




In [11]:
clf = GridSearchCV(pipe, parameters, n_jobs=2, verbose=3, cv=3)
clf.fit(X, y)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 2/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=52;, score=nan total time=   5.3s
[CV 1/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=52;, score=nan total time=   5.5s
[CV 3/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=52;, score=nan total time=   4.8s
[CV 1/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=53;, score=nan total time=   5.2s
[CV 2/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=53;, score=nan total time=   4.4s
[CV 3/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=53;, score=nan total time=   4.9s
[CV 1/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=54;, score=nan total time=   4.4s
[CV 2/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=54;, score=nan total time=   4.7s
[CV 1/3] END logistic_Reg__C=2.0, logistic_

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('std_slc', StandardScaler()),
                                       ('pca', PCA()),
                                       ('logistic_Reg', LogisticRegression())]),
             n_jobs=2,
             param_grid={'logistic_Reg__C': [2.0, 20.0],
                         'logistic_Reg__penalty': ['l1', 'l2'],
                         'pca__n_components': [52, 53, 54, 55, 56, 57, 58, 59]},
             verbose=3)

In [12]:
print('Best Penalty:', clf.best_estimator_.get_params()['logistic_Reg__penalty'])
print('Best C:', clf.best_estimator_.get_params()['logistic_Reg__C'])
print('Best Number Of Components:', clf.best_estimator_.get_params()['pca__n_components'])
print(); print(clf.best_estimator_.get_params()['logistic_Reg'])

Best Penalty: l2
Best C: 2.0
Best Number Of Components: 52

LogisticRegression(C=2.0)


In [14]:
CV_result = cross_val_score(clf , X , y , cv = 3 , n_jobs=2, verbose=3)

# gives error, lamenting something about configuration of models :(

Fitting 3 folds for each of 32 candidates, totalling 96 fits
Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=52;, score=nan total time=   5.5s
[CV 2/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=52;, score=nan total time=   6.1s
[CV 1/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=52;, score=nan total time=   6.7s
[CV 2/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=52;, score=nan total time=   6.7s
[CV 3/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=52;, score=nan total time=   5.7s
[CV 3/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=52;, score=nan total time=   6.7s
[CV 1/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=53;, score=nan total time=   6.0s
[CV 1/3] END logistic_Reg__C=2.0, logistic_Reg__penalty=l1, pca__n_components=53;, score=nan t

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/home/dansolombrino/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dansolombrino/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py", line 875, in fit
    self._run_search(evaluate_candidates)
  File "/home/dansolombrino/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py", line 1379, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "/home/dansolombrino/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py", line 852, in evaluate_

In [ ]:
pca = PCA(n_components= 60  )
pca_features = pca.fit_transform(X)
 
print('Shape before PCA: ', X.shape)
print('Shape after PCA: ', pca_features.shape)

In [ ]:
import matplotlib.pyplot as plt 

plt.rcParams["figure.figsize"] = (12,8)

fig, ax = plt.subplots()
xi = np.arange(1, 61, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='-', color='red')

plt.xlabel('Number of Components')
plt.xticks(np.arange(1, 61, step=1), rotation = 90) 
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.7, color='darkgreen', linestyle='--')
plt.text(1.1, 1, '70% cut-off threshold', color = 'darkgreen', fontsize=16)

ax.grid()
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split( stacked_single_tensors , labels, test_size=0.2, random_state=42 )
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(X_train, Y_train)


In [ ]:
LogisticPredictions = logistic.predict( X_test )
Logistic_Accuracy = logistic.score( X_test , Y_test) 

In [ ]:
LogisticPredictions

In [ ]:
Logistic_Accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix( Y_test , LogisticPredictions , labels = None, sample_weight=None, normalize=None)

In [ ]:
from sklearn.model_selection import KFold
key_fold = KFold( n_splits=4 ) 
key_fold.get_n_splits(X_train)

In [ ]:
import pandas as pd
df = pd.DataFrame(stacked_single_tensors)
X = df.iloc[:,:-1]
y = labels


In [ ]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import utils
from tqdm import tqdm
set.seed(69)


X = df.iloc[:,:-1]

#convert y values to categorical values

y = pd.DataFrame(labels)

#Implementing cross validation

k = 3
kf = KFold(n_splits=k, random_state= None , shuffle= True )
model = LogisticRegression()

acc_score = []

for train_index , test_index in tqdm( kf.split(X) ):

    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y.iloc[train_index] , y.iloc[test_index]


    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score( pred_values , y_test)
    acc_score.append(acc)

avg_acc_score = sum(acc_score)/k
